# Grader:
wrangle_zillow1.py  is the py file created from this exercise
wrangle_mall.py     is the py file created from this exercise

In [1]:
# personally made imports
import env

# typical imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/zillow'

In [3]:
def new_zillow_data():
    return pd.read_sql('''SELECT *
FROM properties_2017 p
LEFT JOIN propertylandusetype t USING (propertylandusetypeid)
LEFT JOIN airconditioningtype a USING (airconditioningtypeid)
LEFT JOIN buildingclasstype b USING (buildingclasstypeid)
LEFT JOIN architecturalstyletype ar USING (architecturalstyletypeid)
RIGHT JOIN predictions_2017 pr USING (parcelid)
WHERE t.propertylandusedesc = 'Single Family Residential'
AND pr.transactiondate LIKE "2017%%"
''', url)


import os

def get_zillow_data():
    filename = "zillow.csv"
    
    # if file is available locally, read it
    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col = 0)
    
    # if file not available locally, acquire data from SQL database
    # and write it as csv locally for future use
    else:
        # read the SQL query into a dataframe
        df_zillow = new_zillow_data()
        
        # Write that dataframe to disk for later. Called "caching" the data for later.
        df_zillow.to_csv(filename)

        # Return the dataframe to the calling code
        return df_zillow

In [4]:
df = new_zillow_data()



Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52441 non-null  int64  
 1   id                            52441 non-null  int64  
 2   logerror                      52441 non-null  float64
 3   transactiondate               52441 non-null  object 
 4   architecturalstyletypeid      70 non-null     float64
 5   buildingclasstypeid           0 non-null      object 
 6   airconditioningtypeid         13638 non-null  float64
 7   propertylandusetypeid         52441 non-null  float64
 8   id                            52441 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52441 non-null  float64
 11  bedroomcnt                    52441 non-null  float64
 12  buildingqualitytypeid         33740 non-null  float64
 13  c

### Notes:
drop redundant or useless columns. Drop id keep parcelid. Drop propertylandusedesc, we already filtered to only include single family residences

In [6]:
df = df.drop(columns = ['id', 'propertylandusedesc'])

In [7]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
52436    False
52437    False
52438    False
52439    False
52440    False
Length: 52441, dtype: bool

In [8]:
df[df.duplicated() == True]

,parcelid,logerror,transactiondate,architecturalstyletypeid,buildingclasstypeid,airconditioningtypeid,propertylandusetypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,buildingclassdesc,architecturalstyledesc


In [9]:
# There are no entire duplicated rows

In [10]:
df[df.parcelid.duplicated() == True]

,parcelid,logerror,transactiondate,architecturalstyletypeid,buildingclasstypeid,airconditioningtypeid,propertylandusetypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,buildingclassdesc,architecturalstyledesc
672,11721753,0.017785,2017-07-21,NaN,None,NaN,261.0,NaN,2.0,3.0,6.0,2.0,NaN,NaN,1316.0,1316.0,None,None,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,7.0,34033180.0,-118355396.0,5672.0,NaN,NaN,NaN,NaN,NaN,0100,LARD1.5,6.037220e+07,12447.0,3101.0,268581.0,95997.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1923.0,NaN,NaN,41948.0,205123.0,2016.0,163175.0,2627.48,None,NaN,6.037220e+13,None,None,None
835,11289917,-0.362001,2017-06-23,NaN,None,1.0,261.0,NaN,2.0,3.0,6.0,2.0,NaN,NaN,1458.0,1458.0,None,None,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34686163.0,-118113100.0,8284.0,1.0,NaN,NaN,NaN,1.0,0101,LRR6000*,6.037901e+07,5534.0,3101.0,NaN,97318.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108890.0,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,Central,None,None
1196,11705026,-0.146056,2017-06-30,NaN,None,NaN,261.0,NaN,1.0,2.0,6.0,1.0,NaN,NaN,1421.0,1421.0,None,None,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,2.0,33999877.0,-118291863.0,6707.0,NaN,NaN,NaN,NaN,NaN,0100,LAR1,6.037232e+07,12447.0,3101.0,118208.0,96018.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1911.0,NaN,NaN,11982.0,35606.0,2016.0,23624.0,543.69,None,NaN,6.037232e+13,None,None,None
1381,14269464,0.021085,2017-06-01,NaN,None,NaN,261.0,NaN,3.0,4.0,NaN,3.0,NaN,NaN,2541.0,2541.0,None,None,NaN,NaN,6059.0,NaN,3.0,2.0,426.0,NaN,NaN,33694636.0,-117912245.0,4975.0,NaN,NaN,NaN,NaN,NaN,1,None,6.059064e+07,38032.0,1286.0,NaN,96958.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,NaN,NaN,434887.0,880456.0,2016.0,445569.0,9819.72,None,NaN,6.059064e+13,None,None,None
1796,11446756,-0.163064,2017-08-23,NaN,None,NaN,261.0,NaN,2.0,3.0,6.0,2.0,NaN,NaN,1491.0,1491.0,None,None,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,7.0,33873860.0,-118338253.0,5000.0,NaN,NaN,NaN,NaN,NaN,0100,TORR-LO,6.037650e+07,54722.0,3101.0,274750.0,96162.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1955.0,NaN,NaN,60537.0,107110.0,2016.0,46573.0,1399.27,None,NaN,6.037650e+13,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35442,12621730,0.098020,2017-08-29,NaN,None,NaN,261.0,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2354.0,2354.0,None,None,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,33807990.0,-118296159.0,6534.0,1.0,NaN,NaN,NaN,1.0,0101,LCR1*,6.037544e+07,118878.0,3101.0,NaN,96210.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1965.0,NaN,NaN,62939.0,97502.0,2016.0,34563.0,5962.64,None,NaN,6.037544e+13,None,None,None
36901,10956664,0.016120,2017-08-31,NaN,None,1.0,261.0,NaN,1.0,2.0,4.0,1.0,NaN,NaN,1178.0,1178.0,None,None,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,2.0,34173900.0,-118346284.0,6515.0,NaN,NaN,NaN,NaN,NaN,0100,BUR1*,6.037311e+07,396054.0,3101.0,NaN,96437.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1943.0,NaN,NaN,32782.0,72522.0,2016.0,39740.0,824.09,None,NaN,6.037311e+13,Central,None,None
38621,14448410,1.648574,2017-08-31,NaN,None,NaN,261.0,NaN,3.0,3.0,NaN,3.0,NaN,NaN,1684.0,1684.0,None,None,Na

In [11]:
#There are 121 rows that have duplicate parcel id's. Looking to deremine what happened. 

#### Attempt 1

In [12]:
l = df[df.parcelid.duplicated() == True]

In [13]:
l;

In [14]:
df.apply(lambda df: df.parcelid == l.parcelid, axis = 1);

In [15]:
#that didnt work, try: 

### Attempt 2

In [16]:
df.sort_values('transactiondate').shape

(52441, 63)

In [17]:
#121 rows are duplicates. before dropping 52279 rows

In [18]:
df = df.sort_values('transactiondate').drop_duplicates('parcelid',keep='last')

In [19]:
df.shape

(52320, 63)

In [20]:
52279 - 52162

117

## Notes:
117 rows dropped. 
possible some parcel ids had more than one duplicate 
(ie four transactions in 2017, there wouuld be three duplicates) 
verify all duplicates are gone

In [21]:
l = df[df.parcelid.duplicated() == True]
l

,parcelid,logerror,transactiondate,architecturalstyletypeid,buildingclasstypeid,airconditioningtypeid,propertylandusetypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,buildingclassdesc,architecturalstyledesc


In [22]:
#all duplicates dropped!

# Acquired continued

In [23]:
# summary stats, info, dtypes, shape, distributions, value_counts, etc

In [24]:
df.describe()

,parcelid,logerror,architecturalstyletypeid,airconditioningtypeid,propertylandusetypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,5.232000e+04,52320.000000,70.00000,13615.000000,52320.0,47.000000,52320.000000,52320.000000,33655.000000,52185.000000,389.0,4371.000000,52239.000000,52074.000000,4371.000000,165.000000,52320.00000,7233.000000,52185.000000,17984.000000,17984.000000,1514.0,33850.000000,5.232000e+04,5.232000e+04,5.195400e+04,11077.0,865.000000,443.0,1071.0,9990.0,5.232000e+04,51284.000000,52320.000000,18981.000000,52294.000000,52320.000000,47.0,6717.000000,76.000000,33762.000000,1935.000000,63.000000,52206.000000,14532.000000,81.0,5.223800e+04,5.231900e+04,52320.0,5.231900e+04,52316.000000,2068.000000,5.219900e+04
mean,1.299682e+07,0.018131,7.10000,2.439589,261.0,678.978723,2.300010,3.300765,6.264894,2.305270,66.0,1546.288035,1923.152702,1925.313899,1567.099291,1241.078788,6049.13555,1.209733,2.240031,1.949066,495.927046,1.0,3.965613,3.402256e+07,-1.181941e+08,1.132335e+04,1.0,518.305202,1.0,1.0,1.0,6.049447e+07,33483.826691,2524.320623,180240.600232,96636.243603,1.894457,7.0,1.013697,5.973684,1.000859,337.359690,225.619048,1963.421254,1.414189,1.0,1.966362e+05,5.298235e+05,2016.0,3.334917e+05,6454.743137,14.101064,6.050245e+13
std,3.350919e+06,0.176905,2.66567,3.847925,0.0,711.825226,1.022807,0.947555,1.716346,1.018067,0.0,675.339282,1004.045419,1003.952951,745.084810,776.959074,21.02913,0.512946,0.993303,0.549974,157.100947,0.0,2.562660,2.740624e+05,3.567149e+05,8.674164e+04,0.0,156.614794,0.0,0.0,0.0,2.086892e+05,49116.149883,804.098580,159928.810868,4610.382493,3.146952,0.0,0.142690,0.229416,0.030290,249.579315,188.505106,23.125670,0.540685,0.0,2.542862e+05,7.518296e+05,0.0,5.705106e+05,8752.477267,2.400399,1.861127e+12
min,1.071186e+07,-4.655420,2.00000,1.000000,261.0,38.000000,0.000000,0.000000,1.000000,1.000000,66.0,184.000000,128.000000,128.000000,184.000000,380.000000,6037.00000,1.000000,1.000000,0.000000,0.000000,1.0,1.000000,3.334062e+07,-1.194754e+08,2.360000e+02,1.0,24.000000,1.0,1.0,1.0,6.037101e+07,3491.000000,1286.000000,6952.000000,95982.000000,0.000000,7.0,1.000000,4.000000,1.000000,11.000000,12.000000,1878.000000,1.000000,1.0,1.290000e+02,1.000000e+03,2016.0,1.610000e+02,49.180000,4.000000,6.037101e+13
25%,1.151018e+07,-0.024707,7.00000,1.000000,261.0,263.500000,2.000000,3.000000,5.000000,2.000000,66.0,1151.000000,1268.000000,1270.000000,1152.000000,784.000000,6037.00000,1.000000,2.000000,2.000000,426.000000,1.0,2.000000,3.382690e+07,-1.184009e+08,5.583000e+03,1.0,424.000000,1.0,1.0,1.0,6.037401e+07,12447.000000,1286.000000,41131.000000,96206.000000,0.000000,7.0,1.000000,6.000000,1.000000,200.000000,71.500000,1950.000000,1.000000,1.0,7.715900e+04,1.940330e+05,2016.0,7.619400e+04,2660.982500,14.000000,6.037401e+13
50%,1.257829e+07,0.006934,7.00000,1.000000,261.0,512.000000,2.000000,3.000000,6.000000,2.000000,66.0,1410.000000,1659.000000,1661.000000,1415.000000,1008.000000,6037.00000,1.000000,2.000000,2.000000,462.000000,1.0,2.000000,3.402375e+07,-1.181531e+08,6.841000e+03,1.0,500.000000,1.0,1.0,1.0,6.037620e+07,24812.000000,3101.000000,118208.000000,96412.000000,0.000000,7.0,1.000000,6.000000,1.000000,280.000000,180.000000,1961.000000,1.000000,1.0,1.319050e+05,3.740060e+05,2016.0,2.180790e+05,4

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52320 entries, 0 to 52440
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52320 non-null  int64  
 1   logerror                      52320 non-null  float64
 2   transactiondate               52320 non-null  object 
 3   architecturalstyletypeid      70 non-null     float64
 4   buildingclasstypeid           0 non-null      object 
 5   airconditioningtypeid         13615 non-null  float64
 6   propertylandusetypeid         52320 non-null  float64
 7   basementsqft                  47 non-null     float64
 8   bathroomcnt                   52320 non-null  float64
 9   bedroomcnt                    52320 non-null  float64
 10  buildingqualitytypeid         33655 non-null  float64
 11  calculatedbathnbr             52185 non-null  float64
 12  decktypeid                    389 non-null    float64
 13  f

In [26]:
df.shape

(52320, 63)

In [27]:
cols=df.columns

In [28]:
#need further cleaning before doing this. going to clean first and come back.

In [29]:
# for col in cols:
#     print('----------------')
#     print(col)
#     col.value_counts()

In [30]:
# for column in df.columns:
#     print(column)
#     print(df[column].value_counts())
#     print("-----------------")

### Aquire continued and prepare

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.


### Prepare

    Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.


### legal definition:  

Single-family residence means any building situated on one lot with a single dwelling, and sharing no common wall, foundation, or other interconnection with another dwelling unit or other structure or use.

In [31]:
# since the legal definition didn't narrow it down I made the following judgment calls:

In [32]:
df = df[(df.bathroomcnt < 11) & (df.bathroomcnt >= 1)]
df = df[(df.bedroomcnt < 11) & (df.bedroomcnt >= 1)]

In [33]:
# maybe try iqr?

## prepare
Create a function that will drop rows or columns based on the percent of values that are missing:

In [34]:
df.isnull().sum()

parcelid                        0
logerror                        0
transactiondate                 0
architecturalstyletypeid    52092
buildingclasstypeid         52162
                            ...  
taxdelinquencyyear          50102
censustractandblock           111
airconditioningdesc         38562
buildingclassdesc           52162
architecturalstyledesc      52092
Length: 63, dtype: int64

In [35]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    prcnt_miss = (num_missing/df.shape[0]) * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [36]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
finishedsquarefeet13,52162,100.000000
buildingclassdesc,52162,100.000000
buildingclasstypeid,52162,100.000000
finishedsquarefeet15,52162,100.000000
basementsqft,52115,99.909896
...,...,...
logerror,0,0.000000
longitude,0,0.000000
latitude,0,0.000000
fips,0,0.000000


In [37]:
df.shape

(52162, 63)

In [38]:
#set threshold to 50% 
#drop all columns that are >50 nulls

In [39]:
threshold = df.shape[0]/2
threshold

26081.0

In [40]:
df = df.dropna(axis=1, thresh=threshold)

In [41]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingqualitytypeid,18538,35.539281
propertyzoningdesc,18479,35.426172
unitcnt,18451,35.372493
heatingorsystemtypeid,18345,35.169280
regionidcity,1028,1.970783
lotsizesquarefeet,352,0.674821
finishedsquarefeet12,166,0.318239
censustractandblock,111,0.212799
structuretaxvaluedollarcnt,72,0.138032
yearbuilt,40,0.076684


### Notes:
it appears to be a sharp cut off at 35% 
set the new cut off to 33.33%

In [42]:
threshold = df.shape[0]/3
threshold

17387.333333333332

In [43]:
52162 - 17387

34775

In [44]:
df = df.dropna(axis=1, thresh = 34775)

In [45]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
regionidcity,1028,1.970783
lotsizesquarefeet,352,0.674821
finishedsquarefeet12,166,0.318239
censustractandblock,111,0.212799
structuretaxvaluedollarcnt,72,0.138032
yearbuilt,40,0.076684
regionidzip,23,0.044093
calculatedbathnbr,16,0.030674
fullbathcnt,16,0.030674
calculatedfinishedsquarefeet,8,0.015337


In [46]:
df.shape

(52162, 27)

# Returnt to Acquire now that data is cleaner

In [47]:
df.columns

Index(['parcelid', 'logerror', 'transactiondate', 'propertylandusetypeid',
       'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidzip', 'roomcnt', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock'],
      dtype='object')

In [48]:
for column in df.columns:
    print(column)
    print(df[column].value_counts())
    print("-----------------")

parcelid
14297519    1
10988846    1
10976437    1
11997371    1
10717871    1
           ..
14347096    1
12251984    1
10912559    1
14064733    1
12826780    1
Name: parcelid, Length: 52162, dtype: int64
-----------------
logerror
 0.000231    5
 0.002652    5
 0.000727    5
 0.001878    5
 0.000420    4
            ..
-0.033509    1
 0.017325    1
-0.000732    1
-0.271602    1
 0.013209    1
Name: logerror, Length: 51904, dtype: int64
-----------------
transactiondate
2017-06-30    806
2017-04-28    612
2017-05-31    564
2017-07-28    544
2017-08-31    528
             ... 
2017-05-13      1
2017-09-02      1
2017-03-25      1
2017-05-07      1
2017-09-25      1
Name: transactiondate, Length: 257, dtype: int64
-----------------
propertylandusetypeid
261.0    52162
Name: propertylandusetypeid, dtype: int64
-----------------
bathroomcnt
2.0     21832
3.0     10646
1.0      9518
2.5      3931
4.0      2225
3.5       916
1.5       839
5.0       801
4.5       686
6.0       319
5.5      

# Return to Prepare now that Acquire is complete

In [49]:
#same thing but change axis so that its rows instead of columns

In [50]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = (num_missing/df.shape[1]) * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

In [51]:
nulls_by_row(df)

,num_cols_missing,percent_cols_missing
10900,5,18.518519
8832,5,18.518519
6809,5,18.518519
34702,4,14.814815
37078,4,14.814815
...,...,...
17688,0,0.000000
17690,0,0.000000
17706,0,0.000000
17704,0,0.000000


In [52]:
df.shape

(52162, 27)

# combine the two functions and set thresholds

In [53]:
def handle_missing_values(df, fraction_required_cols, fraction_required_rows):
    #fraction_required_cols * len(df.index)
    #       take the fraction of required columns and mulitply by the number of rows(the number of values that will need to be required by the column is made up of rows)
    #round(fraction_required_cols * len(df.index), 0)
    #       round to zero decimal places
    #int()
    #       convert to integer
    threshold = int(round(fraction_required_cols * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    
    #absence of row values is reflected in the absence of the nubmer of possible boxes, aka number of columns
    threshold = int(round(fraction_required_rows * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)
    return df

Initial dataset:
52441, 66

Post prep:
52439, 30

In [54]:
# Verify I combined them correctly and get same result

In [55]:
df = new_zillow_data()

In [56]:
df.shape

(52441, 66)

In [57]:
df = handle_missing_values(df, .68, .75)

In [58]:
df.shape

(52439, 30)

In [59]:
df.to_csv('zillow.csv')

# Mall Customers

Notebook

    1. Acquire data from the customers table in the mall_customers database.
    
    2. Summarize the data (include distributions and descriptive statistics).
    
    3. Detect outliers using IQR.
    
    4. Split data into train, validate, and test.
    
    5. Encode categorical columns using a one hot encoder (pd.get_dummies).
    
    6. Handles missing values.
    
    7. Scaling

Encapsulate your work in a wrangle_mall.py python module.

# 1

In [60]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/mall_customers'

In [61]:
def new_mall_data():
    return pd.read_sql('''SELECT *
FROM customers
''', url)


import os

def get_mall_data():
    filename = "mall.csv"
    
    # if file is available locally, read it
    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col = 0)
    
    # if file not available locally, acquire data from SQL database
    # and write it as csv locally for future use
    else:
        # read the SQL query into a dataframe
        df_mall = new_mall_data()
        
        # Write that dataframe to disk for later. Called "caching" the data for later.
        df_mall.to_csv(filename)

        # Return the dataframe to the calling code
        return df_mall

In [83]:
df = new_mall_data()

In [84]:
df.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


# 2
summary stats, info, dtypes, shape, distributions, value_counts,

In [63]:
df.describe()

,customer_id,age,annual_income,spending_score
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [65]:
for column in df.columns:
    print(column)
    print(df[column].value_counts())
    print("-----------------")

customer_id
1      1
138    1
128    1
129    1
130    1
      ..
70     1
71     1
72     1
73     1
200    1
Name: customer_id, Length: 200, dtype: int64
-----------------
gender
Female    112
Male       88
Name: gender, dtype: int64
-----------------
age
32    11
35     9
19     8
31     8
30     7
49     7
40     6
38     6
47     6
27     6
36     6
23     6
34     5
20     5
29     5
50     5
48     5
21     5
24     4
18     4
28     4
67     4
59     4
54     4
43     3
60     3
45     3
39     3
33     3
37     3
22     3
25     3
46     3
68     3
52     2
44     2
66     2
57     2
26     2
53     2
42     2
63     2
70     2
51     2
58     2
65     2
41     2
55     1
69     1
64     1
56     1
Name: age, dtype: int64
-----------------
annual_income
54     12
78     12
48      6
71      6
63      6
       ..
58      2
59      2
16      2
64      2
137     2
Name: annual_income, Length: 64, dtype: int64
-----------------
spending_score
42    8
55    7
46    6
73    6
35    

# 3 outliers 
Typically decide between
z-score: appropriate for normal data (normally distributed)
Tukey IQR method: not contingent on normality

but we are specificallay told in the prompt to use IQR

# IQR

Calculate IQR

    Get Q3 and Q1
    Get difference (q3-q1)
    Establish "fences":
        Standard inner fence: k = 1.5
        Standard outer fence: k = 3.0
        Upper bound: q3 + k * IQR
        Lower bound: q1 - k * IQR
        
Note: 1.5 and 3.0 are standards, but you can make them anything. If you have no clue, those standards are good starting points


In [66]:
df.columns

Index(['customer_id', 'gender', 'age', 'annual_income', 'spending_score'], dtype='object')

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [68]:
df.describe().round(2)

,customer_id,age,annual_income,spending_score
count,200.00,200.00,200.00,200.00
mean,100.50,38.85,60.56,50.20
std,57.88,13.97,26.26,25.82
min,1.00,18.00,15.00,1.00
25%,50.75,28.75,41.50,34.75
50%,100.50,36.00,61.50,50.00
75%,150.25,49.00,78.00,73.00
max,200.00,70.00,137.00,99.00


In [69]:
cols = ['age', 'annual_income']
cols

['age', 'annual_income']

In [70]:
for col in cols:
    print(col)
    q1, q3 = df[col].quantile([0.25, 0.75])
    print(q1, q3)
    print("-----------------")

age
28.75 49.0
-----------------
annual_income
41.5 78.0
-----------------


# 4
split data

In [71]:
def split_mall_data(df):
    train_validate, test = train_test_split(df, test_size=.2, random_state=51)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=51)
    return train, validate, test

In [72]:
train, validate, test = split_mall_data(df)

In [73]:
df.shape, train.shape, validate.shape, test.shape

((200, 5), (112, 5), (48, 5), (40, 5))

# 5 
encode catagorical data

In [74]:
#create dummy columns for catagorical varaibles
dummy_df = pd.get_dummies(df['gender'], dummy_na=False, drop_first= True)
df = pd.concat([df, dummy_df], axis=1)

df = df.drop(columns = 'gender')

In [75]:
df.head()

,customer_id,age,annual_income,spending_score,Male
0,1,19,15,39,1
1,2,21,15,81,1
2,3,20,16,6,0
3,4,23,16,77,0
4,5,31,17,40,0


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   customer_id     200 non-null    int64
 1   age             200 non-null    int64
 2   annual_income   200 non-null    int64
 3   spending_score  200 non-null    int64
 4   Male            200 non-null    uint8
dtypes: int64(4), uint8(1)
memory usage: 6.6 KB


In [77]:
train, validate, test = split_mall_data(df)

In [78]:
x_train = train.drop(columns=['spending_score'])
y_train = train.spending_score

x_validate = validate.drop(columns=['spending_score'])
y_validate = validate.spending_score

x_test = test.drop(columns=['spending_score'])
y_test = test.spending_score

# 6
no missing values

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   customer_id     200 non-null    int64
 1   age             200 non-null    int64
 2   annual_income   200 non-null    int64
 3   spending_score  200 non-null    int64
 4   Male            200 non-null    uint8
dtypes: int64(4), uint8(1)
memory usage: 6.6 KB


# 7 
scaling

In [80]:
from sklearn.preprocessing import MinMaxScaler

#Define function to scale all data based on the train subset
def mms_scale_data(train, validate, test):
    
    mms_cols = ['age', 'annual_income']
    
    train_mms = train.copy()
    validate_mms = validate.copy()
    test_mms = test.copy()
    
    mms = MinMaxScaler()
    
    mms.fit(train[mms_cols])
    
    train_mms[mms_cols] = mms.transform(train[mms_cols])
    validate_mms[mms_cols] = mms.transform(validate[mms_cols])
    test_mms[mms_cols] = mms.transform(test[mms_cols])
    
    return train_mms, validate_mms, test_mms

In [81]:
x_train_mms, x_validate_mms, x_test_mms = mms_scale_data(x_train, x_validate, x_test)

In [82]:
x_train_mms

,customer_id,age,annual_income,Male
38,39,0.346154,0.198198,0
87,88,0.076923,0.378378,0
49,50,0.250000,0.225225,0
22,23,0.538462,0.090090,0
33,34,0.000000,0.162162,1
...,...,...,...,...
34,35,0.596154,0.162162,0
152,153,0.500000,0.567568,0
60,61,1.000000,0.279279,1
111,112,0.019231,0.432432,0
